In [48]:
from pathlib import Path
import time
import datetime
from IPython.display import clear_output

from scipy.io import wavfile
import numpy as np
import pandas as pd
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import keras

from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dense, Input, Dropout, Flatten, Activation, Conv1D, LSTM
from keras.layers.advanced_activations import PReLU
from keras.initializers import Constant
from keras.initializers import he_normal, he_uniform
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

import os
from keras.regularizers import l2
from keras.models import Sequential

from glob import glob

from scipy.fftpack import fft
import re
import gc

from sklearn.model_selection import train_test_split

In [2]:
# seems to reduce the chance of gpu errors, also doesn't blindly allocate all vram 

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)

In [3]:
def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)

def list_wavs_fname(dirpath, ext='wav'):
    fpaths = glob(os.path.join(dirpath, r'*/*' + ext))
    pat = r'.+\\(\w+)\\\w+\.wav$'
    labels = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            labels.append(r.group(1))
    pat = r'.+\\(\w+\.wav)$'
    fnames = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            fnames.append(r.group(1))
    return labels, fnames

def pad_audio(samples):
    if len(samples) >= L: return samples
    else: return np.pad(samples, pad_width=(L - len(samples), 0), mode='constant', constant_values=(0, 0))

def chop_audio(samples, L=16000, num=20):
    for i in range(num):
        beg = np.random.randint(0, len(samples) - L)
        yield samples[beg: beg + L]

def label_transform(labels):
    nlabels = []
    for label in labels:
        if label == '_background_noise_':
            nlabels.append('silence')
        elif label not in legal_labels:
            nlabels.append('unknown')
        else:
            nlabels.append(label)
    return pd.get_dummies(pd.Series(nlabels))

In [4]:
L = 16000
legal_labels = 'yes no up down left right on off stop go silence unknown'.split()

#src folders
root_path = r'.\\data'
out_path = r'.'
model_path = r'.'
train_data_path = os.path.join(root_path, 'input', 'train', 'audio')
test_data_path = os.path.join(root_path, 'input', 'test', 'audio')

In [41]:
labels, fnames = list_wavs_fname(train_data_path)

new_sample_rate = 16000
y = []
x = []

for label, fname in zip(labels, fnames):
    if label not in legal_labels and label != '_background_noise_' and np.random.randint(10) > 0:
        continue
    sample_rate, samples = wavfile.read(os.path.join(train_data_path, label, fname))
    samples = pad_audio(samples)
    if len(samples) > 16000:
        n_samples = chop_audio(samples)
    else: n_samples = [samples]
    for samples in n_samples:
        resampled = signal.resample(samples, int(new_sample_rate / sample_rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        y.append(label)
        x.append(specgram)
x = np.array(x)
y = label_transform(y)
label_index = y.columns.values
y = y.values
y = np.array(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
del x, y
gc.collect()

.\\data\input\train\audio


C:\Users\01124585\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  # Remove the CWD from sys.path while we load stuff.


33

In [52]:
# def cnn_lstm():

#     model = Sequential()
#     model.add(Conv1D(256, 10, strides=4, input_shape=(99, 161)))
#     model.add(BatchNormalization())
#     model.add(Activation('relu'))
#     model.add(Dropout(0.2))
        
#     model.add(LSTM(128, activation='relu', return_sequences=True, dropout=0.2))
#     model.add(LSTM(128, activation='relu', return_sequences=True, dropout=0.2))

#     # 1 fully connected layer DNN ReLu with default 20% dropout
#     model.add(Dense(64))
#     model.add(Activation('relu'))
#     model.add(Dropout(0.2))
    
#     # Output layer with softmax
#     model.add(Dense(12))
#     model.add(Activation('softmax'))
    
#     return model

# model = cnn_lstm()
# model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 23, 256)           412416    
_________________________________________________________________
batch_normalization_2 (Batch (None, 23, 256)           1024      
_________________________________________________________________
activation_4 (Activation)    (None, 23, 256)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 23, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 23, 128)           197120    
_________________________________________________________________
lstm_4 (LSTM)                (None, 23, 128)           131584    
_________________________________________________________________
dense_3 (Dense)              (None, 23, 64)           

In [62]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Bidirectional, TimeDistributed, Conv1D, ZeroPadding1D, GRU
from tensorflow.keras.layers import Lambda, Input, Dropout, Masking, BatchNormalization, Activation
from tensorflow.keras.models import Model

def cnn_lstm():
    input_data = Input(shape=(99, 161))

    x = Conv1D(filters=256, kernel_size=10, strides=4)(input_data)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
        
    x = LSTM(128, activation='relu', return_sequences=True, dropout=0.2)(x)
    x = LSTM(128, activation='relu', return_sequences=False, dropout=0.2)(x)

    x = Dense(units=64, activation='relu')(x)
    x = Dropout(0.2)(x)

    # Output layer with softmax
    y_pred = Dense(units=12, activation='softmax')(x)

    network_model = Model(inputs=input_data, outputs=y_pred)
    
    return network_model

K.clear_session()
model = cnn_lstm()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 99, 161)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 23, 256)           412416    
_________________________________________________________________
batch_normalization (BatchNo (None, 23, 256)           1024      
_________________________________________________________________
activation (Activation)      (None, 23, 256)           0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 256)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 23, 128)           197120    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584

In [69]:
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import TensorBoard

sgd = SGD(lr=0.00001, clipnorm=1.0)
adam = Adam(lr=1e-4, clipnorm=1.0)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128, epochs=40,
                    validation_data=(x_test, y_test))

Train on 22392 samples, validate on 5598 samples
Epoch 1/40
22392/22392 [==============================] - 10s 467us/sample - loss: 0.2270 - accuracy: 0.9273 - val_loss: 0.2587 - val_accuracy: 0.9198
Epoch 2/40
22392/22392 [==============================] - 7s 325us/sample - loss: 0.2093 - accuracy: 0.9320 - val_loss: 0.2648 - val_accuracy: 0.9234
Epoch 3/40
22392/22392 [==============================] - 7s 333us/sample - loss: 0.2163 - accuracy: 0.9306 - val_loss: 0.2454 - val_accuracy: 0.9248
Epoch 4/40
22392/22392 [==============================] - 7s 335us/sample - loss: 0.2237 - accuracy: 0.9270 - val_loss: 0.2488 - val_accuracy: 0.9237
Epoch 5/40
22392/22392 [==============================] - 7s 331us/sample - loss: 0.2010 - accuracy: 0.9332 - val_loss: 0.2674 - val_accuracy: 0.9223
Epoch 6/40
22392/22392 [==============================] - 7s 328us/sample - loss: 0.1953 - accuracy: 0.9361 - val_loss: 0.2571 - val_accuracy: 0.9237
Epoch 7/40
22392/22392 [==========================

In [64]:
import collections
collections.Counter(label_index[np.argmax(y_train, axis=1)])

Counter({'up': 1898,
         'off': 1890,
         'right': 1906,
         'yes': 1901,
         'go': 1888,
         'down': 1885,
         'left': 1885,
         'unknown': 3343,
         'no': 1909,
         'on': 1881,
         'stop': 1910,
         'silence': 96})

In [65]:
def test_data_generator(batch):
    fpaths = glob(os.path.join(test_data_path, '*wav'))
    i = 0
    for path in fpaths:
        if i == 0:
            imgs = []
            fnames = []
        i += 1
        rate, samples = wavfile.read(path)
        samples = pad_audio(samples)
        resampled = signal.resample(samples, int(new_sample_rate / rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        imgs.append(specgram)
        fnames.append(path.split('\\')[-1])
        if i == batch:
            i = 0
            imgs = np.array(imgs)
            yield fnames, imgs
    if i < batch:
        imgs = np.array(imgs)
        yield fnames, imgs
    raise StopIteration()

In [70]:
gc.collect()

index = []
results = []
t0 = datetime.datetime.now()
i = 1
b = 128

for fnames, imgs in test_data_generator(b):
    predicts = model.predict(imgs)
    predicts = np.argmax(predicts, axis=1)
    predicts = [label_index[p] for p in predicts]
    index.extend(fnames)
    results.extend(predicts)
    delta = datetime.datetime.now() - t0
    clear_output(wait=True)
    print('%.2f%% %.1f seconds left' % (b*i/1585.38, delta.total_seconds()*(158538/b/i-1)))
    i+=1

100.03% -0.3 seconds left


RuntimeError: generator raised StopIteration

In [71]:
df = pd.DataFrame(columns=['fname', 'label'])
df['fname'] = index
df['label'] = results
df.to_csv(os.path.join(out_path, 'submission.csv'), index=False)

In [72]:
df['label'].value_counts()

unknown    65841
up         11920
go         11586
silence     8736
no          8168
right       7890
down        7853
on          7481
off         7456
left        7259
yes         7196
stop        7152
Name: label, dtype: int64